In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("example.csv")
df

,name,author,number of ratings,url,avg_rat,Num_pages,Awards,Genre,Publishdate
0,"A Game of Thrones (A Song of Ice and Fire, #1)",George R.R. Martin,"2,046,885",https://www.goodreads.com//book/show/13496.A_G...,4.45,835 pages,"['Nebula Award Nominee for Best Novel (1997)',...","['Fantasy', 'Fiction', 'Fantasy', 'Epic Fantas...","[<nobr class=""greyText"">\n (first..."
1,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit ...,J.R.R. Tolkien,"112,425",https://www.goodreads.com//book/show/30.J_R_R_...,4.60,Hobbit Movie Tie-in Boxed set,[],"['Fantasy', 'Fiction', 'Classics', 'Adventure'...","[<nobr class=""greyText"">\n (first..."
2,The Name of the Wind (The Kingkiller Chronic...,Patrick Rothfuss (Goodreads Author),"739,299",https://www.goodreads.com//book/show/186074.Th...,4.52,662 pages,['Locus Award Nominee for Best First Novel and...,"['Fantasy', 'Fiction', 'Fantasy', 'Epic Fantas...","[<nobr class=""greyText"">\n (first..."
3,"The Way of Kings (The Stormlight Archive, #1)",Brandon Sanderson (Goodreads Author),"319,614",https://www.goodreads.com//book/show/7235533-t...,4.62,1007 pages,['Locus Award Nominee for Best Fantasy Novel (...,"['Fantasy', 'Fiction', 'Fantasy', 'Epic Fantas...",[]
4,The Chronicles of Narnia (Chronicles of Narn...,C.S. Lewis,"532,855",https://www.goodreads.com//book/show/11127.The...,4.26,Reissue Edition,[],"['Fantasy', 'Classics', 'Fiction', 'Young Adul...","[<nobr class=""greyText"">\n (first..."
5,"The Eye of the World (The Wheel of Time, #1)",Robert Jordan,"402,524",https://www.goodreads.com//book/show/228665.Th...,4.17,814 pages,['Locus Award Nominee for Best Fantasy Book (1...,"['Fantasy', 'Fiction', 'Fantasy', 'Epic Fantas...","[<nobr class=""greyText"">\n (first..."
6,"The Final Empire (Mistborn, #1)",Brandon Sanderson (Goodreads Author),"419,345",https://www.goodreads.com//book/show/68428.The...,4.44,544 pages,"['Tähtifantasia Award Nominee (2018)', 'Prix E...","['Fantasy', 'Fiction', 'Fantasy', 'High Fantas...","[<nobr class=""greyText"">\n (first..."
7,His Dark Materials (His Dark Materials #1-3),Philip Pullman,"147,381",https://www.goodreads.com//book/show/18116.His...,4.25,1088 pages,['Literaturpreis der Jury der jungen Leser for...,"['Fantasy', 'Young Adult', 'Fiction', 'Childre...","[<nobr class=""greyText"">\n (first..."
8,"Eragon (The Inheritance Cycle, #1)",Christopher Paolini (Goodreads Author),"1,470,169",https://www.goodreads.com//book/show/113436.Er...,3.91,503 pages,"[""Book Sense Book of the Year Award for Childr...","['Fantasy', 'Young Adult', 'Fiction', 'Fantasy...","[<nobr class=""greyText"">\n (first..."
9,"Assassin's Apprentice (Farseer Trilogy, #1)",Robin Hobb (Goodreads Author),"242,007",https://www.goodreads.com//book/show/77197.Ass...,4.15,435 pages,"['Locus Award Nominee for Fantasy (1996)', 'Br...","['Fantasy', 'Fiction', 'Fantasy', 'Epic Fantas...","[<nobr class=""greyText"">\n (first..."


In [ ]:
# First cell is just preprocessing

In [25]:
import pandas as pd
import numpy as np

def preprocessing():
    df = pd.read_csv("example.csv")
    
    # Calculating number of awards
    df['Awards'] = df['Awards'].str.strip('[]')
    df['Awards'] = df['Awards'].replace(r'^\s*$', np.NaN, regex=True)
    df['Awards_x'] = df.Awards.str.split(',')
    df['award_count'] = df['Awards_x'].str.len()
    df['award_count'] = df['award_count'].fillna(0)
    df['award_count'] = df['award_count'].astype(int)
    df = df.drop(['Awards_x'], axis= 1)
    
    # Cleaning author column from paranthesis text
    df['author'] = df.author.str.replace(r"\(.*Goodreads Author\)","")

    # cleaning number of ratings
    df['number of ratings'] = df['number of ratings'].str.replace('[,]',"")
    df['number of ratings'] = df['number of ratings'].astype(int)
    
    # cleaning avg_rat
    #df['avg_rat'] = df.avg_rat.str.replace('[^0-9.]',"")
    
    # cleaning Genre
    df['Genre'] = df['Genre'].str.strip('[]')
    #df['Genre'] = df['Genre'].replace('[,]', '')
    
    # Cleaning pages column from text
    df['Num_pages'] = df.Num_pages.str.replace('[^0-9]',"")
    df['Num_pages'] = df['Num_pages'].replace(r'^\s*$', np.NaN, regex=True)
    df['Num_pages'] = pd.to_numeric(df['Num_pages'])
    
    # Cleaning publish year
    df['Publishdate'] = df.Publishdate.str.replace('[^0-9]',"")
    df['Publishdate'] = df['Publishdate'].replace(r'^\s*$', np.NaN, regex=True)
    df['original_publish_year'] = df['Publishdate'].str[-4:]
    df['original_publish_year'] = df['original_publish_year'].fillna(-1)
    df['original_publish_year'] = df['original_publish_year'].astype(int)
    df['original_publish_year'] = df['original_publish_year'].replace(-1, np.NaN)
    df['original_publish_year'] = df['original_publish_year'].astype('Int64')
    df = df.drop(['Publishdate'], axis= 1)

    # Calculating normalized ratings
    df['minmax_norm_ratings'] = 1 + (df['avg_rat'] - df.avg_rat.min()) / (df.avg_rat.max()-df.avg_rat.min()) *9
    df['mean_norm_ratings'] = 1 + (df['avg_rat'] - df.avg_rat.mean()) / (df.avg_rat.max()-df.avg_rat.min()) *9
    
    return df

preprocessing()

,name,author,number of ratings,url,avg_rat,Num_pages,Awards,Genre,award_count,original_publish_year,minmax_norm_ratings,mean_norm_ratings
0,"A Game of Thrones (A Song of Ice and Fire, #1)",George R.R. Martin,2046885,https://www.goodreads.com//book/show/13496.A_G...,4.45,835.0,"'Nebula Award Nominee for Best Novel (1997)', ...","'Fantasy', 'Fiction', 'Fantasy', 'Epic Fantasy...",4,1996,7.845070,2.432394
1,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit ...,J.R.R. Tolkien,112425,https://www.goodreads.com//book/show/30.J_R_R_...,4.60,NaN,NaN,"'Fantasy', 'Fiction', 'Classics', 'Adventure',...",0,1955,9.746479,4.333803
2,The Name of the Wind (The Kingkiller Chronic...,Patrick Rothfuss,739299,https://www.goodreads.com//book/show/186074.Th...,4.52,662.0,'Locus Award Nominee for Best First Novel and ...,"'Fantasy', 'Fiction', 'Fantasy', 'Epic Fantasy...",8,2007,8.732394,3.319718
3,"The Way of Kings (The Stormlight Archive, #1)",Brandon Sanderson,319614,https://www.goodreads.com//book/show/7235533-t...,4.62,1007.0,'Locus Award Nominee for Best Fantasy Novel (2...,"'Fantasy', 'Fiction', 'Fantasy', 'Epic Fantasy...",4,<NA>,10.000000,4.587324
4,The Chronicles of Narnia (Chronicles of Narn...,C.S. Lewis,532855,https://www.goodreads.com//book/show/11127.The...,4.26,NaN,NaN,"'Fantasy', 'Classics', 'Fiction', 'Young Adult...",0,1956,5.436620,0.023944
5,"The Eye of the World (The Wheel of Time, #1)",Robert Jordan,402524,https://www.goodreads.com//book/show/228665.Th...,4.17,814.0,'Locus Award Nominee for Best Fantasy Book (19...,"'Fantasy', 'Fiction', 'Fantasy', 'Epic Fantasy...",1,1990,4.295775,-1.116901
6,"The Final Empire (Mistborn, #1)",Brandon Sanderson,419345,https://www.goodreads.com//book/show/68428.The...,4.44,544.0,"'Tähtifantasia Award Nominee (2018)', 'Prix El...","'Fantasy', 'Fiction', 'Fantasy', 'High Fantasy...",2,2006,7.718310,2.305634
7,His Dark Materials (His Dark Materials #1-3),Philip Pullman,147381,https://www.goodreads.com//book/show/18116.His...,4.25,1088.0,'Literaturpreis der Jury der jungen Leser for ...,"'Fantasy', 'Young Adult', 'Fiction', 'Children...",1,2000,5.309859,-0.102817
8,"Eragon (The Inheritance Cycle, #1)",Christopher Paolini,1470169,https://www.goodreads.com//book/show/113436.Er...,3.91,503.0,"""Book Sense Book of the Year Award for Childre...","'Fantasy', 'Young Adult', 'Fiction', 'Fantasy'...",20,2002,1.000000,-4.412676
9,"Assassin's Apprentice (Farseer Trilogy, #1)",Robin Hobb,242007,https://www.goodreads.com//book/show/77197.Ass...,4.15,435.0,"'Locus Award Nominee for Fantasy (1996)', 'Bri...","'Fantasy', 'Fiction', 'Fantasy', 'Epic Fantasy...",3,1995,4.042254,-1.370423


In [9]:
def analyse_highest_book(df, a):
    dp = df.groupby(df.original_publish_year).agg({'minmax_norm_ratings': np.mean})
    dp = dp.rename(columns={'minmax_norm_ratings': 'minmax_norm_ratings_mean'})
    author_group = df.loc[df['author'] == a]
    book_max_rating = author_group.iloc[author_group['minmax_norm_ratings'].argmax()]['name']
    print(dp)
    return book_max_rating

print(analyse_highest_book(preprocessing(), 'Robert Jordan'))

                       minmax_norm_ratings_mean
original_publish_year                          
1955                                   9.746479
1956                                   5.436620
1990                                   4.295775
1995                                   4.042254
1996                                   7.845070
2000                                   5.309859
2002                                   1.000000
2006                                   7.718310
2007                                   8.732394
  The Eye of the World (The Wheel of Time, #1)


In [27]:
# working on this one

def analyse_highest_book(df, a):
    df2 = df.dropna(subset=['original_publish_year'])
    df2["yeargrp"] = pd.cut(df2.original_publish_year, [1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020])
    dp = df2.groupby(df2.yeargrp).agg({'minmax_norm_ratings': np.mean})
    dp = dp.rename(columns={'minmax_norm_ratings': 'minmax_norm_ratings_mean'})
    
    author_group = df.loc[df['author'] == a]
    book_max_rating = author_group.iloc[author_group['minmax_norm_ratings'].argmax()]['name']
    print(dp)
    return book_max_rating

print(analyse_highest_book(preprocessing(), 'Robert Jordan'))

              minmax_norm_ratings_mean
yeargrp                               
(1910, 1920]                       NaN
(1920, 1930]                       NaN
(1930, 1940]                       NaN
(1940, 1950]                       NaN
(1950, 1960]                  7.591549
(1960, 1970]                       NaN
(1970, 1980]                       NaN
(1980, 1990]                  4.295775
(1990, 2000]                  5.732394
(2000, 2010]                  5.816901
(2010, 2020]                       NaN
  The Eye of the World (The Wheel of Time, #1)


<ipython-input-27-e72a15b583fe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["yeargrp"] = pd.cut(df2.original_publish_year, [1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020])
